In [5]:

# library
import pandas as pd
import numpy as np

#plotting 
from plotnine import *

#api calls
from dotenv import load_dotenv

#webscraping
import requests
import os
import time
from bs4 import BeautifulSoup

#selenium
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

### MERGING

In [6]:
#reading in files
nyt = pd.read_csv("../data/nyt_results.csv")
nyp = pd.read_csv("../data/nyp_results.csv")
nydn = pd.read_csv("../data/nydn_results.csv")
chalkbeat = pd.read_csv("../data/chalkbeat_results.csv")
amny = pd.read_csv("../data/amny_results.csv")
spectrum = pd.read_csv("../data/spectrum_results.csv")
brooklyn = pd.read_csv("../data/brooklyn_results.csv")
gotham = pd.read_csv("../data/gotham_results.csv")
queens = pd.read_csv("../data/queens_results.csv")

In [7]:
#add source column
nyt["source"] = "NY Times"
nyp["source"] = "NY Post"
nydn["source"] = "NY Daily News"
chalkbeat["source"] = "Chalkbeat"
amny["source"] = "AMNY"
spectrum["source"] = "Spectrum"
brooklyn["source"] = "Brooklyn Daily Eagle"
gotham["source"] = "Gotham Gazette"
queens["source"] = "Queens Chronicle"

In [8]:
#extracting year when possible
nyt["year"] = nyt["date_published"].str[0:4]
nyp["year"] = [date.split(",")[1].split(" ")[1] for date in nyp["date_published"].tolist()]
nydn["year"] = nydn["date_published"].str[0:4]
chalkbeat["year"] = chalkbeat["date_published"].str[-4:]
#amny["year"] = amny["date_published"].str[-4:]
amny["year"] = [year if not pd.isna(year) else link.split(".com/")[1].split("/")[0]for year, link in zip(amny["date_published"].str[-4:], amny["link"])]
spectrum["year"] = spectrum["date_published"][0][-19:][6:10]
gotham["year"] = [date[-4:] if len(date) == 12 or len(date) == 11 else np.nan for date in [date.split(" ...")[0] for date in gotham["date_published"].tolist()]]
brooklyn["year"] = [np.nan if pd.isna(date) else date[:4] for date in [ np.nan if pd.isna(date) or "/articles/" not in date else date.split("/articles/")[1] for date in brooklyn["link"].tolist()]]
queens["year"] = queens["date_published"].str[0:4]

In [9]:
test = pd.concat([nyt, nyp, nydn, amny, chalkbeat, spectrum, brooklyn, gotham, queens]).drop_duplicates().query("~link.isna()")

### NOT NYT Times

In [18]:
test = requests.get(nydn["link"].iloc[1])

In [23]:
BeautifulSoup(test.content, 'html.parser').get_text()

' \n\n\n \n\n\nReaders sound off on Trump\'s DOJ, rights of the accused and elite high schools\n\n\n\n/* <![CDATA[ */\nvar scriptHelpers = {"staticPath":"https:\\/\\/www.nydailynews.com\\/wp-content\\/themes\\/assets\\/static\\/"};\n/* ]]> */\n\nbody{--primary:#3f51b5;--secondary:#eeff41;--tertiary:#303f9f;--primary-light:#5869ca;--primary-dark:#32408f;--primary-darker:#171e44;--font-primary:"Noto Serif",Helvetica,serif;--font-secondary:"Noto Sans",Helvetica,sans-serif}:root{--primary:#006fb7;--secondary:#d70000;--tertiary:#ffc518;--primary-lighter:#47b6ff;--primary-light:#0093f5;--primary-dark:#00568f;--primary-darker:#003d66;--site-branding-primary:#00102e;--site-branding-secondary:#233452;--site-branding-tertiary:#3d5268;--site-branding-gray-100:#38322a;--site-branding-gray-200:#5d5b5a;--site-branding-gray-300:#9d9a98;--site-branding-gray-400:#aaa7a4;--site-branding-gray-500:#d7d6d2;--site-branding-gray-600:#f1efeb;--primary-obit:#8d092d;--primary-obit-light:#c00c3d;--secondary-obit

### TESTING

In [ ]:
load_dotenv()
nyt_email = os.environ.get('nyt_email')
nyt_user = os.environ.get('nyt_user')
nyt_pw = os.environ.get('nyt_pw')

In [ ]:
nyt_only = test.query("source ==  'NY Times'")

In [ ]:
login_url = "https://myaccount.nytimes.com/auth/enter-email?response_type=cookie&client_id=lgcl&redirect_uri=https%3A%2F%2Fwww.nytimes.com"

In [ ]:
query = nyt_only["link"].iloc[0]

In [ ]:
driver = uc.Chrome()

In [ ]:
driver.get(login_url)

In [ ]:
email_field = driver.find_element(By.ID, "email")
email_field.send_keys(nyt_email)
email_field.send_keys(Keys.ENTER)

In [ ]:
continue_button = driver.find_element(By.ID, "js-google-oauth-login")
continue_button.click()

In [ ]:
main = driver.current_window_handle
driver.switch_to.window(driver.window_handles[1])

In [ ]:
email_field = driver.find_element(By.ID, "identifierId")

In [ ]:
email_field.send_keys(nyt_email)


In [ ]:
email_field.send_keys(Keys.ENTER)

In [ ]:
username = driver.find_element(By.ID, "username")
username.send_keys(nyt_user)

In [ ]:
pw = driver.find_element(By.ID, "password")
pw.send_keys(nyt_pw)

In [ ]:
pw.send_keys(Keys.ENTER)
#pause to authenticate

In [ ]:
trust_button = driver.find_element(By.ID, "trust-browser-button")
trust_button.click()

In [ ]:
driver.switch_to.window(driver.window_handles[2])

In [ ]:
continue_button = driver.find_element(By.XPATH, "//span[contains(., 'Continue')]/parent::button")